In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from os import listdir, makedirs
from os.path import join, isfile, isdir, exists
from datetime import datetime
import pandas as pd
import numpy as np
import math
from tqdm import tqdm

In [ ]:
# Input
root_folder = '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe'
ActiPAL_15s_epoch_proc_folder = '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/Data/processed_data/ActiPAL/epoch_data/15s_ref'
ActiPAL_20hz_raw_folder = '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe/activPAL_raw'

# sleep time input_data
sleep_time_file = '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/Data/metadata/record_availability_final_pal.csv'

# Output
ActiPAL_output_folder = '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/processed data/montoye_thigh'

### Load Sleep Data

In [ ]:
raw_files = [f for f in listdir(ActiPAL_20hz_raw_folder) if isfile(join(ActiPAL_20hz_raw_folder, f))]
epoch_files = [f for f in listdir(ActiPAL_15s_epoch_proc_folder) if isfile(join(ActiPAL_15s_epoch_proc_folder, f))]

data_dict = {f.split('-')[0]: {'raw': join(ActiPAL_20hz_raw_folder, f)} for f in raw_files}
for ef in epoch_files:
  key = ef.split('-')[0]
  if key in data_dict.keys():
    data_dict[key]['epoch'] = join(ActiPAL_15s_epoch_proc_folder, ef)

In [ ]:
print(len(epoch_files))

165


In [ ]:
sleep_df = pd.read_csv(sleep_time_file)
sleep_df.head(2)
print(len(sleep_df))

1367


In [ ]:
completed_folder = '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/processed data/montoye_thigh/Epoch30'
completed_files = [f for f in listdir(completed_folder) if isfile(join(completed_folder, f))]
completed_ids = []
for k in range(len(completed_files)):
  completed_ids.append(completed_files[k].split('_')[0])
  # print(completed_files[k].split('_')[0])
completed_ids = list(set(completed_ids))
expected_ids = []
# print(list(set(completed_ids)))
for _, row in tqdm(sleep_df.iterrows(), total=sleep_df.shape[0]):
  user_id = row['Participant']
  begin_time = row['begin_time_pal']
  end_time = row['end_time_pal']
  output_file_name = '{}_{}_to_{}_{}.csv'.format(user_id, begin_time, end_time, epoch_name).replace(':', '-').replace(' ', '_')
  expected_ids.append(output_file_name)
  # print(output_file_name)
# print(expected_ids)

left_ids = []
for k in expected_ids:
    if(k not in completed_files):
      left_ids.append(k)
# expected_ids.sort()
# completed_files.sort()
print(left_ids)
print(len(left_ids))
# print(completed_files[:20])
print(len(completed_files))
# print(len(raw_files))
# print(len(epoch_files))

# print(data_dict['HOA1'])


100%|██████████| 1367/1367 [00:00<00:00, 8965.90it/s]

['HOA11_2019-07-01_13-59-47_to_2019-07-01_20-00-00_Epoch30.csv', 'HOA11_2019-07-02_06-00-00_to_2019-07-02_20-00-00_Epoch30.csv', 'HOA11_2019-07-03_06-00-00_to_2019-07-03_20-00-00_Epoch30.csv', 'HOA11_2019-07-04_06-00-00_to_2019-07-04_20-00-00_Epoch30.csv', 'HOA11_2019-07-05_06-00-00_to_2019-07-05_20-00-00_Epoch30.csv', 'HOA11_2019-07-06_06-00-00_to_2019-07-06_20-00-00_Epoch30.csv', 'HOA11_2019-07-07_06-00-00_to_2019-07-07_20-00-00_Epoch30.csv', 'HOA11_2019-07-08_06-00-00_to_2019-07-08_11-09-17_Epoch30.csv', 'HOA12_2019-07-01_15-59-47_to_2019-07-01_20-00-00_Epoch30.csv', 'HOA12_2019-07-02_05-00-00_to_2019-07-02_20-00-00_Epoch30.csv', 'HOA12_2019-07-03_05-00-00_to_2019-07-03_20-00-00_Epoch30.csv', 'HOA12_2019-07-04_05-00-00_to_2019-07-04_20-00-00_Epoch30.csv', 'HOA12_2019-07-05_05-00-00_to_2019-07-05_20-00-00_Epoch30.csv', 'HOA12_2019-07-06_05-00-00_to_2019-07-06_20-00-00_Epoch30.csv', 'HOA12_2019-07-07_05-00-00_to_2019-07-07_20-00-00_Epoch30.csv', 'HOA12_2019-07-08_05-00-00_to_2019-07-0

### Generate statistical features

In [ ]:
FREQUENCY = 20 #HZ amount of the accelerometer
REFERENCE_EPOCH = 15 #Epoch value that EE is given
TIME_EPOCH_DICT = {
    # 'Epoch1': FREQUENCY * 1,
    # 'Epoch5': FREQUENCY * 5,
    'Epoch30': FREQUENCY * 30,
    # 'Epoch60': FREQUENCY * 60
}

In [ ]:
for _, row in tqdm(sleep_df.iterrows(), total=sleep_df.shape[0]):
  user_id = row['Participant']
  print(_,user_id)
  # if(user_id in completed_ids):continue
  try:
    begin_time = row['begin_time_pal']
    end_time = row['end_time_pal']

    age = row['age_yr']
    sex = row['sex']
    BMI = row['BMI']
    mass = row['bodymass_kg']

    height = (mass/BMI)**0.5

    # if('HOA12' != user_id):
    #   continue
    # print(user_id)

    # get respective epoch dataset
    epoch_df = pd.read_csv(data_dict[user_id]['epoch'])
    epoch_df = epoch_df[(begin_time < epoch_df['Time_stamp']) & (epoch_df['Time_stamp'] < end_time)]
    # print(list(epoch_df.index)[0])
    # print(epoch_df.head())
    
    # continue
    # get respective raw dataset
    raw_begin_index = list(epoch_df.index)[0] * REFERENCE_EPOCH * FREQUENCY
    raw_data_length = epoch_df.shape[0] * REFERENCE_EPOCH * FREQUENCY
    raw_df_orig = pd.read_csv(data_dict[user_id]['raw'], skiprows=raw_begin_index,sep = ';', nrows=raw_data_length,usecols=[0,2,3,4], header=None)
    raw_df_orig.columns = ['Timestamp', 'X', 'Y', 'Z']
    # print(raw_df_orig.head(20))
    
    raw_df_orig['X_sc'] = 0.0157723997930183*raw_df_orig['X'] - 2.01571269354774
    raw_df_orig['Y_sc'] = 0.0157723997930183*raw_df_orig['Y'] - 2.01571269354774
    raw_df_orig['Z_sc'] = 0.0157723997930183*raw_df_orig['Z'] - 2.01571269354774
    # print(raw_df_orig.head(20))
    # break


    """
    Process RAW data as required by the model features
    """

    # VM and MANGLE needs for calculations by Staudenmayer model
    # Calculate the vector magnitude from X, Y, Z raw readings
    # Calculate the angle of arcsin from X and VM, arcsin(axis used/vector magnitude)/(pi/2)
    raw_df_orig['vm'] = np.sqrt([(raw_df_orig.X_sc ** 2) + (raw_df_orig.Y_sc ** 2) + (raw_df_orig.Z_sc ** 2)])[0]
    # raw_df_orig['angle'] = (90 * np.arcsin(raw_df_orig.X / raw_df_orig['vm'])) / (math.pi / 2)
    # print(raw_df_orig['vm'].head(20))
    # Calculate ENMO (Euclidean Norm Minus One) reguired for Hilderbrand model
    # raw_df_orig['enmo'] = np.sqrt([(raw_df_orig.X ** 2) + (raw_df_orig.Y ** 2) + (raw_df_orig.Z ** 2)])[0] - 1
    # raw_df_orig.loc[raw_df_orig.enmo < 0, 'enmo'] = 0
    # print(raw_df_orig['enmo'].head(20))


    orientation_roll = np.arctan2(raw_df_orig.Y_sc, raw_df_orig.Z_sc) * 180/math.pi
    orientation_pitch = np.arctan2(raw_df_orig.X_sc,np.sqrt((raw_df_orig.Z_sc*raw_df_orig.Z_sc)+(raw_df_orig.Y_sc*raw_df_orig.Y_sc)))*180/math.pi
    angle = (90 * np.arcsin(raw_df_orig.X_sc / raw_df_orig['vm'])) / (math.pi / 2)

    raw_df_orig['orientation_roll'] = orientation_roll
    raw_df_orig['orientation_pitch'] = orientation_pitch
    raw_df_orig['angle'] = angle


    
    # For each epoch given
    for epoch_name, epoch_duration in TIME_EPOCH_DICT.items():
      print(epoch_name,epoch_duration)
      
      # continue
      # Create output path for the combined file
      output_file_name = join(ActiPAL_output_folder, epoch_name,'{}_{}_to_{}_{}.csv'.format(user_id, begin_time, end_time, epoch_name).replace(':', '-').replace(' ', '_'))
      if(output_file_name not in left_ids):continue
      if not exists(join(ActiPAL_output_folder, epoch_name)):
        makedirs(join(ActiPAL_output_folder, epoch_name))
      
      # Only run for non processed files
      # if exists(output_file_name):
      #   continue

      # Copy raw data
      raw_df = raw_df_orig.copy()
      # print(raw_df.head(10))
      # print(raw_df.index)
      raw_df.reset_index(drop=True, inplace=True)
      # print(raw_df.head(10))
      # print(raw_df.index)
      

      # X shifted needs to calculate covariance for Montoye 2017 model
      raw_df['X_shifted'] = raw_df_orig['X_sc'].shift(epoch_duration)
      raw_df['Y_shifted'] = raw_df_orig['Y_sc'].shift(epoch_duration)
      raw_df['Z_shifted'] = raw_df_orig['Z_sc'].shift(epoch_duration)

      """
      Calculate the statistical inputs (Features)
      """
      # print(raw_df.index // epoch_duration)
      
      hip_grouped_temp = raw_df.groupby(raw_df.index // epoch_duration)
      # k=0
      # for name,group in hip_grouped_temp:
      #   print(name)
      #   print(group)
      #   k=k+1
      #   if(k==100):break
      # break
      aggregated_hip = pd.DataFrame()

      # General stats
      aggregated_hip['mvm'] = hip_grouped_temp['vm'].mean()

      # # Aggregated values required for Hilderbrand model
      # aggregated_hip['enmo'] = hip_grouped_temp['enmo'].mean()
      # aggregated_hip['enmo'] = aggregated_hip['enmo'].fillna(1)

      # Aggregated values required for Montoye Models
      aggregated_hip['XMean'] = hip_grouped_temp['X_sc'].mean()
      aggregated_hip['YMean'] = hip_grouped_temp['Y_sc'].mean()
      aggregated_hip['ZMean'] = hip_grouped_temp['Z_sc'].mean()


      aggregated_hip['XVar'] = hip_grouped_temp['X_sc'].var()
      aggregated_hip['YVar'] = hip_grouped_temp['Y_sc'].var()
      aggregated_hip['ZVar'] = hip_grouped_temp['Z_sc'].var()

      aggregated_hip['orientation_roll_mean'] = hip_grouped_temp['orientation_roll'].mean()
      aggregated_hip['orientation_roll_var'] = hip_grouped_temp['orientation_roll'].var()

      aggregated_hip['orientation_pitch_mean'] = hip_grouped_temp['orientation_pitch'].mean()
      aggregated_hip['orientation_pitch_var'] = hip_grouped_temp['orientation_pitch'].var()

      aggregated_hip['angle_mean'] = hip_grouped_temp['angle'].mean()
      aggregated_hip['angle_var'] = hip_grouped_temp['angle'].var()
      

      aggregated_hip['X10perc'] = hip_grouped_temp['X_sc'].quantile(.1)
      aggregated_hip['X25perc'] = hip_grouped_temp['X_sc'].quantile(.25)
      aggregated_hip['X50perc'] = hip_grouped_temp['X_sc'].quantile(.5)
      aggregated_hip['X75perc'] = hip_grouped_temp['X_sc'].quantile(.75)
      aggregated_hip['X90perc'] = hip_grouped_temp['X_sc'].quantile(.9)

      aggregated_hip['Y10perc'] = hip_grouped_temp['Y_sc'].quantile(.1)
      aggregated_hip['Y25perc'] = hip_grouped_temp['Y_sc'].quantile(.25)
      aggregated_hip['Y50perc'] = hip_grouped_temp['Y_sc'].quantile(.5)
      aggregated_hip['Y75perc'] = hip_grouped_temp['Y_sc'].quantile(.75)
      aggregated_hip['Y90perc'] = hip_grouped_temp['Y_sc'].quantile(.9)

      aggregated_hip['Z10perc'] = hip_grouped_temp['Z_sc'].quantile(.1)
      aggregated_hip['Z25perc'] = hip_grouped_temp['Z_sc'].quantile(.25)
      aggregated_hip['Z50perc'] = hip_grouped_temp['Z_sc'].quantile(.5)
      aggregated_hip['Z75perc'] = hip_grouped_temp['Z_sc'].quantile(.75)
      aggregated_hip['Z90perc'] = hip_grouped_temp['Z_sc'].quantile(.9)

      aggregated_hip['X_cov'] = hip_grouped_temp.apply(lambda x: x['X_sc'].cov(x['X_shifted']))
      aggregated_hip['Y_cov'] = hip_grouped_temp.apply(lambda x: x['Y_sc'].cov(x['Y_shifted']))
      aggregated_hip['Z_cov'] = hip_grouped_temp.apply(lambda x: x['Z_sc'].cov(x['Z_shifted']))

      aggregated_hip['X_cov'] = aggregated_hip['X_cov'].fillna(0)
      aggregated_hip['Y_cov'] = aggregated_hip['Y_cov'].fillna(0)
      aggregated_hip['Z_cov'] = aggregated_hip['Z_cov'].fillna(0)

      aggregated_hip['X_min'] = hip_grouped_temp['X_sc'].min()
      aggregated_hip['Y_min'] = hip_grouped_temp['Y_sc'].min()
      aggregated_hip['Z_min'] = hip_grouped_temp['Z_sc'].min()
      aggregated_hip['X_max'] = hip_grouped_temp['X_sc'].max()
      aggregated_hip['Y_max'] = hip_grouped_temp['Y_sc'].max()
      aggregated_hip['Z_max'] = hip_grouped_temp['Z_sc'].max()

      """
      Repeat rows of epoch file as required to supplement raw data.
      """
      # Number of rows for each epoch
      epoch_df_current = epoch_df.copy()
      # print(epoch_df_current.head())
      epoch_df_current.reset_index(drop=True, inplace=True)
      # print('kk',epoch_df_current.head())
      
      multiplication_factor = float(REFERENCE_EPOCH / (epoch_duration/FREQUENCY))#ex: 60/(1800/60) for 30 epochs, ultimately 60/30
      # print('mul',[multiplication_factor,REFERENCE_EPOCH,epoch_duration,FREQUENCY])
      # print('before',epoch_df_current.head())
      # print('befor_len',len(epoch_df_current))
      # epoch_df_current = epoch_df_current.reindex(np.repeat(epoch_df_current.index.values, multiplication_factor), method='ffill')
      print('before',len(epoch_df_current))
      print(epoch_df_current.head(20))

      #for 30s epoch from 15s
      ground_thigh = []
      lim = int(len(epoch_df_current)/2)
      for i in range(0,len(epoch_df_current),2):

        mean_ee = epoch_df_current['thigh_EE'].iloc[i:i+2].mean()
        ground_thigh.append(mean_ee)
      #   std_w_e = epoch_df_current['enmo'].iloc[i:i+10].std()

      print('pros',len(ground_thigh))

      epoch_df_current = epoch_df_current.iloc[::2]
      print('after',len(epoch_df_current.iloc[::2]))
      epoch_df_current['thigh_ee_final'] = ground_thigh
      print(epoch_df_current.head(20))



      # print('EE',epoch_df_current['waist_ee'])

      epoch_df_current.reset_index(drop=True, inplace=True)
      # print('after',epoch_df_current.head())
      # print('after_len',len(epoch_df_current))
      # break
      # print("test",[len(epoch_df_current),len(aggregated_hip['XMean'])])

  
      
      # break




    
      
      # epoch_df_current['enmo'] = aggregated_hip['enmo']


      # Montoye models and caron models
      #1 mean
      epoch_df_current['raw_XMean'] = aggregated_hip['XMean']
      epoch_df_current['raw_YMean'] = aggregated_hip['YMean']
      epoch_df_current['raw_ZMean'] = aggregated_hip['ZMean']

      #2 variance
      epoch_df_current['raw_XVar'] = aggregated_hip['XVar']
      epoch_df_current['raw_YVar'] = aggregated_hip['YVar']
      epoch_df_current['raw_ZVar'] = aggregated_hip['ZVar']

      #3 covariance
      epoch_df_current['raw_X_cov'] = aggregated_hip['X_cov']
      epoch_df_current['raw_Y_cov'] = aggregated_hip['Y_cov']
      epoch_df_current['raw_Z_cov'] = aggregated_hip['Z_cov']

      #4 minimum
      epoch_df_current['raw_Xmin'] = aggregated_hip['X_min']
      epoch_df_current['raw_Ymin'] = aggregated_hip['Y_min']
      epoch_df_current['raw_Zmin'] = aggregated_hip['Z_min']

      #5 maximum
      epoch_df_current['raw_Xmax'] = aggregated_hip['X_max']
      epoch_df_current['raw_Ymax'] = aggregated_hip['Y_max']
      epoch_df_current['raw_Zmax'] = aggregated_hip['Z_max']

      #6.1 orientation_roll
      epoch_df_current['raw_orientation_roll_mean'] = aggregated_hip['orientation_roll_mean']
      #7.1 orientation_roll
      epoch_df_current['raw_orientation_roll_var']  = aggregated_hip['orientation_roll_var']

      #6.2 orientation_pitch
      epoch_df_current['raw_orientation_pitch_mean'] = aggregated_hip['orientation_pitch_mean']
      #7.2 orientation_pitch
      epoch_df_current['raw_orientation_pitch_var']  = aggregated_hip['orientation_pitch_var']

      #6.3 angle
      epoch_df_current['raw_angle_mean'] = aggregated_hip['angle_mean']
      #7.3 angle
      epoch_df_current['raw_angle_var']  = aggregated_hip['angle_var']

      #8-12 percentiles
      epoch_df_current['raw_X10perc'] = aggregated_hip['X10perc']
      epoch_df_current['raw_X25perc'] = aggregated_hip['X25perc']
      epoch_df_current['raw_X50perc'] = aggregated_hip['X50perc']
      epoch_df_current['raw_X75perc'] = aggregated_hip['X75perc']
      epoch_df_current['raw_X90perc'] = aggregated_hip['X90perc']

      epoch_df_current['raw_Y10perc'] = aggregated_hip['Y10perc']
      epoch_df_current['raw_Y25perc'] = aggregated_hip['Y25perc']
      epoch_df_current['raw_Y50perc'] = aggregated_hip['Y50perc']
      epoch_df_current['raw_Y75perc'] = aggregated_hip['Y75perc']
      epoch_df_current['raw_Y90perc'] = aggregated_hip['Y90perc']

      epoch_df_current['raw_Z10perc'] = aggregated_hip['Z10perc']
      epoch_df_current['raw_Z25perc'] = aggregated_hip['Z25perc']
      epoch_df_current['raw_Z50perc'] = aggregated_hip['Z50perc']
      epoch_df_current['raw_Z75perc'] = aggregated_hip['Z75perc']
      epoch_df_current['raw_Z90perc'] = aggregated_hip['Z90perc']

      #vm for carons
      epoch_df_current['mvm'] = aggregated_hip['mvm']

      #user_characteristics
      epoch_df_current['sex'] = sex
      epoch_df_current['height'] = height
      epoch_df_current['age'] = age
      epoch_df_current['BMI'] = BMI
      epoch_df_current['mass'] = mass

      # print('epoch_df', epoch_df.shape)
      # print('epoch_df_current', epoch_df_current.shape, multiplication_factor)
      # print('raw', aggregated_hip.shape)
      # print('ag_hip', aggregated_hip.head())
      # print('after',epoch_df_current.head())

      epoch_df_current.to_csv(output_file_name, index=None)
  except SyntaxError:
    print()

  # except Exception as e:
  #   f = open(ActiPAL_output_folder+'/errors.txt','w+')
  #   f.write(str(user_id)+" "+str(e)+" \n")
  #   f.close()




  0%|          | 0/1367 [00:00<?, ?it/s]

0 HOA1
1 HOA1
2 HOA1
3 HOA1
4 HOA1
5 HOA1
6 HOA1
7 HOA1
8 HOA10
9 HOA10
10 HOA10
11 HOA10
12 HOA10
13 HOA10
14 HOA10
15 HOA10
16 HOA11
17 HOA11
18 HOA11
19 HOA11
20 HOA11
21 HOA11
22 HOA11
23 HOA11
24 HOA12


KeyError: ignored

In [ ]:
data = pd.DataFrame()
data['X'] = [1,2,3,4,5,6,7,8,9]
data['Y'] = [0,2,4,6,8,10,12,14,16]
data['Z'] = [3,6,9,12,15,18,21,24,27]

grouped = data.groupby(data.index // 2)
new_d = data.iloc[::2]
new_d['X_sum'] = grouped['X'].sum()
print(data)
# print(grouped)
print(new_d)
# print(mean_g)

   X   Y   Z
0  1   0   3
1  2   2   6
2  3   4   9
3  4   6  12
4  5   8  15
5  6  10  18
6  7  12  21
7  8  14  24
8  9  16  27
   X   Y   Z  X_sum
0  1   0   3    3.0
2  3   4   9   11.0
4  5   8  15    9.0
6  7  12  21    NaN
8  9  16  27    NaN


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
